In [1]:
#Dependencies
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import statistics
import seaborn as sns

In [2]:
#Read CSV and store into Pandas df
load_file = "Resources\shopify_data1.csv"
shopify = pd.read_csv(load_file)
shopify.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [3]:
#Examine column names
shopify.columns

Index(['order_id', 'shop_id', 'user_id', 'order_amount', 'total_items',
       'payment_method', 'created_at'],
      dtype='object')

In [4]:
#Examine data types
shopify.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB


In [6]:
#Examine shape
shopify.shape

(5000, 7)

In [7]:
#Statistics of 'order_amount' and 'total_items'
shopify[['order_amount','total_items']].describe()

,order_amount,total_items
count,5000.000000,5000.00000
mean,3145.128000,8.78720
std,41282.539349,116.32032
min,90.000000,1.00000
25%,163.000000,1.00000
50%,284.000000,2.00000
75%,390.000000,3.00000
max,704000.000000,2000.00000


In [11]:
#Check for nulls
shopify.isnull().sum()

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
created_at        0
dtype: int64

## Analysis on AOV

In [14]:
#Filter for only necessary columns: 'shop_id', 'order_amount', 'total_items'
shoes = shopify.copy()
shoes = shoes[['shop_id','order_amount','total_items']]

In [16]:
#Even out the weight of purchases to total_items and create a new column
shoes['single_order'] = shoes['order_amount']/shoes['total_items']
shoes.head()

,shop_id,order_amount,total_items,single_order
0,53,224,2,112.0
1,92,90,1,90.0
2,44,144,1,144.0
3,18,156,1,156.0
4,18,156,1,156.0


In [17]:
#Calculate for AOV
np.mean(shoes['single_order'])

387.7428

AOV still seems to be considered higher than it should be. There may be stores increasing the price.

In [18]:
#Examine stores selling price per shoe
shoes.groupby(shoes['shop_id'])['single_order'].mean().reset_index().sort_values(by=['single_order'], ascending=False).head()

,shop_id,single_order
77,78,25725.0
41,42,352.0
11,12,201.0
88,89,196.0
98,99,195.0


Store 78 is the outlier causing the average to be higher.

In [19]:
#Remove store 78 to get an accurate average
np.mean(shoes[shoes['shop_id'] !=78]['single_order'])

152.47557529269278

Conclusion: The sneaker is actually affordable across 99 stores at an average price of ~$152.48. The cause of the higher average was due to store 78 selling the sneaker at a price of $25,725.00. 

## Re-evaluating the AOV

In [20]:
shoes2 = shopify.copy()

In [21]:
#What the number of orders are at each amount of total items
shoes2['total_items'].value_counts()

2       1832
1       1830
3        941
4        293
5         77
2000      17
6          9
8          1
Name: total_items, dtype: int64

In [22]:
#See what store is selling 2000 items per order
shoes2[shoes2['total_items'] == 2000]['shop_id'].unique()

array([42], dtype=int64)

Store 42 is selling 2000 items per order... but is that a store or customer issue?

In [23]:
shoes2 = shoes2[['shop_id','user_id','order_amount','total_items']]
shoes2[shoes2['shop_id'] == 42].head()

,shop_id,user_id,order_amount,total_items
15,42,607,704000,2000
40,42,793,352,1
60,42,607,704000,2000
308,42,770,352,1
409,42,904,704,2


In [24]:
shoes2[shoes2['user_id'] == 607].head()

,shop_id,user_id,order_amount,total_items
15,42,607,704000,2000
60,42,607,704000,2000
520,42,607,704000,2000
1104,42,607,704000,2000
1362,42,607,704000,2000


user_id 607 seems to be cause of the anomalies and thus consider to remove store 42 because consumer spending is an uncontrollable factor in our analysis. 

In [26]:
#Create the filtered dataset with store 42 and 78 excluded
shopify_filtered = shopify[(shopify['shop_id'] != 42) & (shopify['shop_id'] != 78)]

In [27]:
#Average amount per order with new df
np.mean(shopify_filtered['order_amount'])

300.1558229655313

In [28]:
#Average items per order with new df
np.mean(shopify_filtered['total_items'])

1.9957169080155006

Conclusion: Average order amount is ~$300.16 while some customers will purchase more than 1 product per visit.

## New Metric

In [29]:
shoes3 = shopify.copy()

In [30]:
shoes3['order_amount'].describe().to_frame().round(2)

,order_amount
count,5000.00
mean,3145.13
std,41282.54
min,90.00
25%,163.00
50%,284.00
75%,390.00
max,704000.00


Here we notice:
* Standard Dev. is a large value which indicates a large difference in order amount.

In [33]:
#Calculate Z-scores using values from the describe() table
shoes3['z_score'] = (shoes3['order_amount'] - 3145.13) / 41282.54
shoes3['z_score'] = shoes3['z_score'].round(4)
shoes3.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,z_score
0,1,53,746,224,2,cash,2017-03-13 12:36:56,-0.0708
1,2,92,925,90,1,cash,2017-03-03 17:38:52,-0.0740
2,3,44,861,144,1,cash,2017-03-14 4:23:56,-0.0727
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37,-0.0724
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11,-0.0724


Initial z_score observations seem normal however earlier in the analysis we saw that store 42 and 78 were anomalies.

In [34]:
shoes3[shoes3['z_score'] > 0.1].head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,z_score
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00,16.9770
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00,16.9770
160,161,78,990,25725,1,credit_card,2017-03-12 5:56:57,0.5470
490,491,78,936,51450,2,debit,2017-03-26 17:08:19,1.1701
493,494,78,983,51450,2,cash,2017-03-16 21:39:35,1.1701


In [35]:
shoes3[abs(shoes3['z_score']) > 0.1]['shop_id'].unique()

array([42, 78], dtype=int64)

Here again we determine that store 78 and 42 are anomalies but instead this metric used is Z_score which had an observation of each independent order. 